In [8]:
import subprocess
import random
import os
import pandas as pd
import numpy as np
import tqdm
import shutil

In [9]:
MACH = 0.5
MACH_VARIABILITY = 0.01

ALPHAS = [5, 6]
ALPHA_VARIABILITY = 0.1

NUM_SIMS = 100

OUTPUT_FILE = "./outputs"

In [10]:
def run_xfoil_commands(command_list):
    """
    Run a batch of XFOIL commands using subprocess and return the output.
    """
    # Join the list of commands into a single string with each command separated by a newline
    commands = "\n".join(command_list)
    
    process = subprocess.Popen(['./XFOIL6.99/xfoil'],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True
                               )

    output, error = process.communicate(input=commands)
    return output

In [11]:
def create_commands(alpha):
    output_path = f"{OUTPUT_FILE}/temp"
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)

    machs = []
    alphas = []

    machs = [round(random.uniform(MACH - MACH_VARIABILITY, MACH + MACH_VARIABILITY), 3) for _ in range(NUM_SIMS)]
    alphas = [round(random.uniform(alpha - ALPHA_VARIABILITY, alpha + ALPHA_VARIABILITY), 3) for _ in range(NUM_SIMS)]

    np.save(f"{output_path}/machs.npy", machs)
    np.save(f"{output_path}/alphas.npy", alphas)

    commands = [
        "PLOP",
        "G F",
        "", 
        "NACA 0012",
        "OPER",
        "ITER 400",
        "Re 250000",
        "VISC"
    ]

    for i in range(NUM_SIMS):
        new_commands = [
            f"MACH {machs[i]}",
            "PACC",
            f"{output_path}/{i}.txt",
            "",
            f"ALFA {alphas[i]}",
            "PACC",
            "PDEL",
            "0"
        ]
        commands.extend(new_commands)
    
    return commands, machs, alphas

In [12]:
def read_output(file_path):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file, start=1):
            if i == 13:  # Check if it's the 13th line
                values = line.split()
                alpha = float(values[0])
                cl = float(values[1])
                cd = float(values[2])
                cdp = float(values[3])
                cm = float(values[4])
                top_xtr = float(values[5])
                bot_xtr = float(values[6])
                return alpha, cl, cd, cdp, cm, top_xtr, bot_xtr
    return None  # Return None if the file doesn't have 13 lines

In [13]:
def process_folder(machs, alphas, folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            result = read_output(file_path)
            if result:
                alpha, cl, cd, cdp, cm, top_xtr, bot_xtr = result
                index = int(os.path.splitext(filename)[0])
                mach = machs[index]
                if alpha != alphas[index]: print("shit")
                data.append([alpha, mach, cl, cd, cdp, cm, top_xtr, bot_xtr])
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Alpha', 'Mach', 'CL', 'CD', 'CDp', 'CM', 'Top_Xtr', 'Bot_Xtr'])
    return df

In [14]:
for alpha in ALPHAS:
    folder_path = f"{OUTPUT_FILE}/temp"
    commands, machs, alphas = create_commands(alpha)
    output = run_xfoil_commands(commands)
    df = process_folder(machs, alphas, folder_path)
    shutil.rmtree(folder_path)
    df.to_csv(f"{OUTPUT_FILE}/{alpha}.csv", index=True)